In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
import os

from astropy.table import Table
import astropy.io.fits as fits

from scipy.optimize import curve_fit

from glob import glob

matplotlib.rcParams.update({'font.size':18})
matplotlib.rcParams.update({'font.family':'serif'})

ftype = '.pdf'

from astropy.stats import LombScargle
from stuff import FINDflare

import exoplanet as xo

## 1. Get the data

In [ ]:
# tess_dir = '/data/epyc/data/tess/'
tess_dir = '/Users/james/Desktop/tess/'

sectors = ['sector001', 'sector002', 'sector003', 'sector004', 'sector005', 'sector006']

# just in case glob wants to re-order things, be sure grab them in Sector order
sect1 = glob(tess_dir + sectors[0] + '/*.fits', recursive=True)
sect2 = glob(tess_dir + sectors[1] + '/*.fits', recursive=True)
sect3 = glob(tess_dir + sectors[2] + '/*.fits', recursive=True)
sect4 = glob(tess_dir + sectors[3] + '/*.fits', recursive=True)
sect5 = glob(tess_dir + sectors[4] + '/*.fits', recursive=True)
sect6 = glob(tess_dir + sectors[5] + '/*.fits', recursive=True)

files = sect1 + sect2 + sect3 + sect4 + sect5 + sect6
# make into an array for looping later!
s_lens = [len(sect1), len(sect2), len(sect3), len(sect4), len(sect5), len(sect6)]
print(s_lens, len(files))

## 2. Run Period/Flare search on the data

In [ ]:
%%time

for i in range(len(sectors)):
    print('running ' + sectors[i])
    
    files_i = glob(tess_dir + sectors[i] + '/*.fits', recursive=True)

    per_out = np.zeros(len(files)) -1 
    per_amp = np.zeros(len(files)) -1 
    per_med = np.zeros(len(files)) -1 
    per_std = np.zeros(len(files)) -1 

    ACF_1pk = np.zeros(len(files)) -1 
    ACF_1dt = np.zeros(len(files)) -1 

    FL_id = np.array([])
    FL_t0 = np.array([]) 
    FL_t1 = np.array([])
    FL_f0 = np.array([])
    FL_f1 = np.array([])
    
    if not os.path.isdir('figures/' + sectors[i]):
        os.makedirs('figures/' + sectors[i])

    for k in range(len(files_i)):
        tbl = -1
        df_tbl = -1
        try:
            tbl = Table.read(files_i[k], format='fits')
            df_tbl = tbl.to_pandas()

            # make harsh quality cuts, and chop out a known bad window of time
            AOK = (tbl['QUALITY'] == 0) & ((tbl['TIME'] < 1347) | (tbl['TIME'] > 1350))

            # require at least 1000 good datapoints
            if sum(AOK) > 1000:
                # do a running median for a basic smooth
                smo = df_tbl['PDCSAP_FLUX'][AOK].rolling(128, center=True).median()
                med = np.nanmedian(smo)

                # find OK points in the smoothed LC
                SOK = np.isfinite(smo)

                FL = FINDflare((df_tbl['PDCSAP_FLUX'][AOK][SOK] - smo[SOK])/med, 
                               df_tbl['PDCSAP_FLUX_ERR'][AOK][SOK]/med, 
                               N1=4, N2=2, N3=5, avg_std=False)

                LS = LombScargle(df_tbl['TIME'][AOK][SOK], smo[SOK]/med, dy=df_tbl['PDCSAP_FLUX_ERR'][AOK][SOK]/med)
                frequency, power = LS.autopower(minimum_frequency=1./40., 
                                                maximum_frequency=1./0.1, 
                                                samples_per_peak=7)
                best_frequency = frequency[np.argmax(power)]

                per_out[k] = 1./best_frequency
                per_amp[k] = np.nanmax(power)
                per_med[k] = np.nanmedian(power)
                per_std[k] = np.nanstd(smo[SOK]/med)


                # ACF w/ Exoplanet package
                acf = xo.autocorr_estimator(tbl['TIME'][AOK][SOK], smo[SOK]/med, 
                                            yerr=tbl['PDCSAP_FLUX_ERR'][AOK][SOK]/med, 
                                            min_period=0.1, max_period=40, max_peaks=2)
                try:
                    ACF_1dt[k] = acf['peaks'][0]['period']
                    ACF_1pk[k] = acf['autocorr'][1][np.where((acf['autocorr'][0] == acf['peaks'][0]['period']))[0]][0]
                except:
                    ACF_1dt[k] = -1
                    ACF_1pk[k] = -1

                if np.size(FL) > 0:
                    for i in range(len(FL[0])):
                        FL_id = np.append(FL_id, k)
                        FL_t0 = np.append(FL_t0, FL[0][i])
                        FL_t1 = np.append(FL_t1, FL[1][i])
                        FL_f0 = np.append(FL_f0, med)
                        FL_f1 = np.append(FL_f1, np.nanmax(tbl['PDCSAP_FLUX'][AOK][SOK][(FL[0][i]):(FL[1][i]+1)]))

                # make an output plot for every file
#                 if (k % 100 == 0):
#                 print('fig. k = '+str(k))

                plt.figure(figsize=(12,9))

                plt.errorbar(tbl['TIME'][AOK], tbl['PDCSAP_FLUX'][AOK]/med, yerr=tbl['PDCSAP_FLUX_ERR'][AOK]/med, 
                             linestyle=None, alpha=0.25, label='PDC_FLUX')
                plt.plot(tbl['TIME'][AOK], smo/med, label='128pt MED')

                Smed = np.nanmedian(tbl['SAP_FLUX'][AOK])
                plt.errorbar(tbl['TIME'][AOK], tbl['SAP_FLUX'][AOK]/Smed, yerr=tbl['SAP_FLUX_ERR'][AOK]/Smed, 
                             linestyle=None, alpha=0.25, label='SAP_FLUX')

                if np.size(FL) > 0:
                    for i in range(len(FL[0])):
                        plt.scatter(tbl['TIME'][AOK][SOK][(FL[0][i]):(FL[1][i]+1)], 
                                    tbl['PDCSAP_FLUX'][AOK][SOK][(FL[0][i]):(FL[1][i]+1)] / med, color='r', 
                                    label='_nolegend_')

                if np.nanmax(power) > 0.2:
                    LSmodel = LS.model(df_tbl['TIME'][AOK][SOK], best_frequency)
                    plt.plot(df_tbl['TIME'][AOK][SOK], LSmodel, 
                             label='L-S P='+format(1./best_frequency, '6.3f')+'d, pk='+format(np.nanmax(power), '6.3f'))

                if ACF_1dt[k] > 0:
                    plt.plot(tbl['TIME'][AOK][SOK], 
                             np.nanstd(smo[SOK]/med) * ACF_1pk[k] * np.sin(tbl['TIME'][AOK][SOK] / ACF_1dt[k] * 2 * np.pi) + 1,
                             label = 'ACF=' + format(ACF_1dt[k], '6.3f') + 'd, pk=' + format(ACF_1pk[k], '6.3f'))

                plt.title(files[k].split('/')[-1] + ' k='+str(k), fontsize=12)
                plt.ylabel('Flux')
                plt.xlabel('BJD - 2457000 (days)')
                plt.legend(fontsize=10)
                plt.savefig('figures/' + sector[i] + '/' + files_i[k].split('/')[-1] + '.jpeg', 
                            bbox_inches='tight', pad_inches=0.25, dpi=100)
                plt.close()

        except (OSError, KeyError, TypeError, ValueError):
            print('k=' + str(k) + ' bad file: ' + files[k])

    # write per-sector output files
    ALL_TIC = pd.Series(files_i).str.split('-', expand=True).iloc[:,-3].astype('int')

    flare_out = pd.DataFrame(data={'TIC':ALL_TIC[FL_id], 'i0':FL_t0, 'i1':FL_t1, 'med':FL_f0, 'peak':FL_f1})
    flare_out.to_csv(sectors[i] + '_flare_out.csv')

    rot_out = pd.DataFrame(data={'TIC':ALL_TIC, 
                                 'per':per_out, 'Pamp':per_amp, 'Pmed':per_med, 'StdLC':per_std, 
                                 'acf_pk':ACF_1pk, 'acf_per':ACF_1dt})
    rot_out.to_csv(sectors[i] + '_rot_out.csv')

In [ ]:
stopthecode

## need to re-work...
analysis plots, etc. Needs to be reworked.

there *should* be some per-sector output plots being made, but they can be made in post

In [ ]:
# plt.figure(figsize=(12,6))

# s1 = np.where((FL_id < len(sect1)))[0]
# s2 = np.where((FL_id > len(sect1)) & 
#               (FL_id < len(sect1) + len(sect2)))[0]
# s3 = np.where((FL_id > len(sect1) + len(sect2)) & 
#               (FL_id < len(sect1) + len(sect2) + len(sect3)))[0]
# s4 = np.where((FL_id > len(sect1) + len(sect2) + len(sect4)))[0]

# _ = plt.hist((FL_t0[s1]+FL_t1[s1])/2., bins=150, alpha=0.5, label='sector001')
# _ = plt.hist((FL_t0[s2]+FL_t1[s2])/2., bins=150, alpha=0.5, label='sector002')
# _ = plt.hist((FL_t0[s3]+FL_t1[s3])/2., bins=150, alpha=0.5, label='sector003')
# _ = plt.hist((FL_t0[s4]+FL_t1[s4])/2., bins=150, alpha=0.5, label='sector004')

# plt.plot([0, max(FL_t1)], [25,25], lw=5, alpha=0.35, c='k', label='bad times')

# plt.title(str(np.size(FL_t0))+' total flare candidates',fontsize=12)
# plt.xlabel('epoch number')
# plt.ylabel('# flare candidates')
# plt.legend(fontsize=12)
# plt.yscale('log')
# plt.savefig('flare_hist.jpeg', dpi=150, bbox_inches='tight', pad_inches=0.25)

In [ ]:
# redo above, but w/ loop now
plt.figure(figsize=(12,6))

for i in range(len(s_lens)):
    si = np.where((FL_id >= np.cumsum([0]+s_lens[0:i+1])[i]) & 
                  (FL_id < np.cumsum([0]+s_lens[0:i+1])[i+1]))
    
    _ = plt.hist((FL_t0[si]+FL_t1[si])/2., bins=150, alpha=0.5, label='sector'+str(i+1))

plt.plot([0, max(FL_t1)], [25,25], lw=5, alpha=0.35, c='k', label='bad times')

plt.title(str(np.size(FL_t0))+' total flare candidates',fontsize=12)
plt.xlabel('epoch number')
plt.ylabel('# flare candidates')
plt.legend(fontsize=12)
plt.yscale('log')
plt.savefig('flare_hist.jpeg', dpi=150, bbox_inches='tight', pad_inches=0.25)

In [ ]:
ALL_TIC = pd.Series(files).str.split('-', expand=True).iloc[:,-3].astype('int')

flare_out = pd.DataFrame(data={'TIC':ALL_TIC[FL_id], 'i0':FL_t0, 'i1':FL_t1, 'med':FL_f0, 'peak':FL_f1})
flare_out.to_csv('flare_out_v04.csv')

rot_out = pd.DataFrame(data={'TIC':ALL_TIC, 
                             'per':per_out, 'Pamp':per_amp, 'Pmed':per_med, 'StdLC':per_std, 
                             'acf_pk':ACF_1pk, 'acf_per':ACF_1dt})
rot_out.to_csv('rot_out_v04.csv')

In [ ]:
# NOW MATCH OUTPUT TO GAIA FILE, PLOT ROTATION ON THE CMD, AND PROT VS COLOR
# could I be looping over this? Probably, yes... meh

df1 = pd.read_csv(tess_dir + 'gaiatess1_xmatch_1arsec-result.csv')
df2 = pd.read_csv(tess_dir + 'gaiatess2_xmatch_1arsec-result.csv')
df3 = pd.read_csv(tess_dir + 'gaiatess3_xmatch_1arsec-result.csv')
df4 = pd.read_csv(tess_dir + 'gaiatess4_xmatch_1arsec-result.csv')
df5 = pd.read_csv(tess_dir + 'gaiatess5_xmatch_1arsec-result.csv')
df6 = pd.read_csv(tess_dir + 'gaiatess6_xmatch_1arsec-result.csv')

df = pd.concat([df1, df2, df3, df4, df5, df6], 
               ignore_index=True, sort=False)

plt.figure(figsize=(9,8))
_ = plt.hist2d(df['ra'], df['dec'], bins=(200,200), cmap=plt.cm.magma_r)
_ = plt.hist2d(df['ra']-360, df['dec'], bins=(200,200), cmap=plt.cm.magma_r)
plt.xlim(-180,180)
plt.xlabel('RA [deg]')
plt.ylabel('Dec [deg]')



In [ ]:
gdata = pd.merge(df, rot_out, right_on='TIC', left_on='ticid')

In [ ]:
# gdata.columns.values

In [ ]:
ok = np.where(np.isfinite(gdata[u'parallax']) & # this is basically the same as the TGAS file...
              (gdata[u'parallax_error'] / gdata[u'parallax'] < 0.1) &
              (gdata[u'modality_flag'] == 1) & 
              (gdata[u'result_flag'] == 1) &
              np.isfinite(gdata[u'bp_rp']) & 
              (gdata[u'phot_bp_mean_flux_error']/gdata[u'phot_bp_mean_flux'] < 0.01) & 
              (gdata[u'phot_rp_mean_flux_error']/gdata[u'phot_rp_mean_flux'] < 0.01) & 
              (gdata[u'phot_g_mean_flux_error']/gdata[u'phot_g_mean_flux'] < 0.01) & 
              (gdata['per'] > 0.1) & 
              (gdata['Pamp']/gdata['Pmed'] > 200) & 
              (gdata['Pamp'] > 0.5))[0]
print(ok.shape)

In [ ]:
plt.figure(figsize=(12,7))
plt.scatter(gdata['bp_rp'].values[ok], gdata['per'].values[ok], s=15, alpha=0.2, c='k')
plt.yscale('log')
plt.ylabel('LS Period (days)')
plt.xlabel('$G_{BP} - G_{RP}$ (mag)')
plt.title('TESS Sectors 1-4', fontsize=12)
plt.savefig('color_LSper6.jpeg', dpi=150, bbox_inches='tight', pad_inches=0.25)

In [ ]:
plt.figure(figsize=(12,7))
plt.scatter(gdata['bp_rp'].values[ok], gdata['acf_per'].values[ok], s=15, alpha=0.2, c='k')
plt.yscale('log')
plt.ylabel('LS Period (days)')
plt.xlabel('$G_{BP} - G_{RP}$ (mag)')
plt.title('TESS Sectors 1-4', fontsize=12)
plt.savefig('color_ACFper6.jpeg', dpi=150, bbox_inches='tight', pad_inches=0.25)

In [ ]:
plt.scatter(per_out, acf_1dt)

In [ ]:
# plt.figure(figsize=(12,7))
# plt.scatter(gdata['bp_rp'].values[ok], gdata['per'].values[ok], s=9, alpha=0.1, c='k')
# plt.yscale('log')
# plt.xlim(0.3,2.3)
# plt.ylabel('LS Period (days)')
# plt.xlabel('$G_{BP} - G_{RP}$ (mag)')
# plt.title('TESS Sectors 1-4', fontsize=12)
# # plt.savefig('color_per1.jpeg', dpi=150, bbox_inches='tight', pad_inches=0.25)

In [ ]:
plt.figure(figsize=(12,7))


# plt.scatter(gdata['bp_rp'].values[ok], gdata['per'].values[ok], s=4, alpha=0.1, c='k')
cb,xeb,yeb,ib = plt.hist2d(gdata['bp_rp'].values[ok], np.log10(gdata['per'].values[ok]),
                           range=[[0,2.2],[-.7,1.3]], bins=(75,70), cmap=plt.cm.Spectral_r)
# plt.yscale('log')
# plt.xlim(0.3,2.3)
plt.ylabel('LS Period (days)')
plt.xlabel('$G_{BP} - G_{RP}$ (mag)')
plt.title('TESS Sectors 1-4', fontsize=12)
# plt.savefig('color_per1.jpeg', dpi=150, bbox_inches='tight', pad_inches=0.25)

In [ ]:
xcb,xeb,yeb,ib = plt.hist2d(gdata['bp_rp'], 
                           gdata['phot_g_mean_mag'] - 5. * np.log10(gdata[u'r_est'].values) + 5, 
                           range=[[-1,4],[-5,15]], bins=(75,75))

In [ ]:

plt.figure(figsize=(9,9))


plt.contour(xcb.T, extent=[xeb.min(),xeb.max(),yeb.min(),yeb.max()], levels=[2,100], colors='k', alpha=0.25)

plt.scatter(gdata['bp_rp'][ok], gdata['phot_g_mean_mag'][ok] - 5. * np.log10(gdata[u'r_est'].values[ok]) + 5, 
            alpha=0.6, s=9, c=(gdata['per'].values[ok]), cmap=plt.cm.Spectral_r)

cbar = plt.colorbar()
cbar.set_label('LS Period (days)')

plt.xlabel('$G_{BP} - G_{RP}$ (mag)')
plt.ylabel('$M_G$ (mag)')
plt.title('TESS Sectors 1-4', fontsize=12)
plt.ylim(15,-5)

# plt.savefig('cmd_per1.jpeg', dpi=150, bbox_inches='tight', pad_inches=0.25)

In [ ]:
msbad = np.where((gdata['phot_g_mean_mag'][ok] - 5. * np.log10(gdata[u'r_est'].values[ok]) + 5 > 5) & 
                 (gdata['phot_g_mean_mag'][ok] - 5. * np.log10(gdata[u'r_est'].values[ok]) + 5 < 13) & 
                 (gdata['bp_rp'][ok] > 0.5))[0]
print(np.shape(msbad))

In [ ]:

plt.figure(figsize=(9,9))

plt.scatter(gdata['bp_rp'][ok[msbad]], gdata['phot_g_mean_mag'][ok[msbad]] - 5. * np.log10(gdata[u'r_est'].values[ok[msbad]]) + 5, 
            alpha=0.6, s=9, c=(gdata['per'].values[ok[msbad]]), cmap=plt.cm.Spectral_r)

cb = plt.colorbar()
cb.set_label('LS Period (days)')
plt.xlabel('$G_{BP} - G_{RP}$ (mag)')
plt.ylabel('$M_G$ (mag)')
# plt.title('TESS Sectors 1,2', fontsize=12)
plt.ylim(10.2,4)
# plt.xlim(1,2)

# plt.savefig('cmd_per2.jpeg', dpi=150, bbox_inches='tight', pad_inches=0.25)

In [ ]:


fig, (a0, a1) = plt.subplots(ncols=1, nrows=2, sharex=True, 
                             gridspec_kw = {'height_ratios':[1,3]},
                             figsize=(7,9))

a0.scatter(gdata['bp_rp'].values[ok[msbad]], np.log10(gdata['per'].values[ok[msbad]]), s=2, alpha=0.2, c='k')
# a0.set_yscale('log')
a0.set_ylabel('log LS Period (days)')
# plt.xlabel('$G_{BP} - G_{RP}$ (mag)')
# plt.xlim(0.67,1.7)
a0.set_ylim(0, 1.3)
a0.set_title('TESS Sectors 1-4', fontsize=12)


a1.scatter(gdata['bp_rp'][ok], gdata['phot_g_mean_mag'][ok] - 5. * np.log10(gdata[u'r_est'].values[ok]) + 5, 
            alpha=0.6, s=9, c=(gdata['per'].values[ok]), cmap=plt.cm.Spectral_r)

# cbar = plt.colorbar()
# cbar.set_label('LS Period (days)')

a1.set_xlabel('$G_{BP} - G_{RP}$ (mag)')
a1.set_ylabel('$M_G$ (mag)')
a1.set_ylim(15,-5)

fig.subplots_adjust(hspace=0)

# plt.savefig('cmd_per1.jpeg', dpi=150, bbox_inches='tight', pad_inches=0.25)

In [ ]:
plt.figure(figsize=(12,7))
plt.scatter(gdata['bp_rp'].values[ok[msbad]], gdata['per'].values[ok[msbad]], s=15, alpha=0.2, c='k')
plt.yscale('log')
plt.ylabel('LS Period (days)')
plt.xlabel('$G_{BP} - G_{RP}$ (mag)')
plt.xlim(0.67,1.7)
plt.ylim(1,30)
# plt.title('TESS Sectors 1,2', fontsize=12)
plt.savefig('color_per2_4.jpeg', dpi=150, bbox_inches='tight', pad_inches=0.25)

In [ ]:
bc1 = np.where((gdata['ra'][ok] > 0) & 
               (gdata['ra'][ok] < 2.2) & 
               (gdata['dec'][ok] > -31) & 
               (gdata['dec'][ok] < 29))[0]
print(np.shape(bc1))

In [ ]:

plt.figure(figsize=(12,7))
plt.scatter(gdata['bp_rp'].values[ok[bc1]], gdata['per'].values[ok[bc1]], alpha=0.5, c='k')
plt.yscale('log')
plt.ylabel('LS Period (days)')
plt.xlabel('$G_{BP} - G_{RP}$ (mag)')
plt.title('Blanco 1', fontsize=12)
# plt.savefig('color_per3.jpeg', dpi=150, bbox_inches='tight', pad_inches=0.25)

In [ ]:
# s4

In [ ]:
NBINS=200


plt.figure(figsize=(9,7))
# s1 = np.where((FL_id < len(sect1)))[0]
# s2 = np.where((FL_id > len(sect1)) & 
#               (FL_id < len(sect1) + len(sect2)))[0]
# s3 = np.where((FL_id > len(sect1) + len(sect2)))[0]


n1,be1,_ = plt.hist((FL_t0[s1]+FL_t1[s1])/2.,bins=NBINS, alpha=0.5, label='sector001')
n2,be2,_ = plt.hist((FL_t0[s2]+FL_t1[s2])/2.,bins=NBINS, alpha=0.5, label='sector002')
n3,be3,_ = plt.hist((FL_t0[s3]+FL_t1[s3])/2.,bins=NBINS, alpha=0.5, label='sector003')
n4,be4,_ = plt.hist((FL_t0[s4]+FL_t1[s4])/2.,bins=NBINS, alpha=0.5, label='sector004')



plt.title(str(np.size(FL_t0))+' flare candidates',fontsize=12)
plt.xlabel('epoch number')
plt.ylabel('# stars')
plt.legend(fontsize=12)
plt.yscale('log')


In [ ]:
print(10**np.mean(np.log10(n1)) + 2 * 10**np.std(np.log10(n1)))
print(10**np.mean(np.log10(n2)) + 2 * 10**np.std(np.log10(n2)))
print(10**np.mean(np.log10(n3)) + 2 * 10**np.std(np.log10(n3)))
print(10**np.mean(np.log10(n4)) + 2 * 10**np.std(np.log10(n4)))

In [ ]:
# need to loopify this

FLCUT = 25

OK = np.empty(len(FL_id), dtype='bool') * False
for k in range(len(n1)):
    tt = np.where(#(FL_id[s1] < 15889) & 
                  ((FL_t0[s1]+FL_t1[s1])/2. >= be1[k]) & 
                  ((FL_t0[s1]+FL_t1[s1])/2. < be1[k+1]))[0]
    if n1[k] < FLCUT:
        OK[s1[tt]] = True


for k in range(len(n2)):
    tt = np.where(#(FL_id[s2] >= 15889) & 
                  ((FL_t0[s2]+FL_t1[s2])/2. >= be2[k]) & 
                  ((FL_t0[s2]+FL_t1[s2])/2. < be2[k+1]))[0]
    if n2[k] < FLCUT:
        OK[s2[tt]] = True

        
for k in range(len(n3)):
    tt = np.where(#(FL_id[s3] >= 15889) & 
                  ((FL_t0[s3]+FL_t1[s3])/2. >= be3[k]) & 
                  ((FL_t0[s3]+FL_t1[s3])/2. < be3[k+1]))[0]
    if n3[k] < FLCUT:
        OK[s3[tt]] = True

        
for k in range(len(n4)):
    tt = np.where(#(FL_id[s3] >= 15889) & 
                  ((FL_t0[s4]+FL_t1[s4])/2. >= be4[k]) & 
                  ((FL_t0[s4]+FL_t1[s4])/2. < be4[k+1]))[0]
    if n4[k] < FLCUT:
        OK[s4[tt]] = True


        
sum(OK)

In [ ]:
plt.figure(figsize=(9,7))
# s1 = np.where((FL_id < 15889))[0]
# s2 = np.where((FL_id >= 15889))[0]

_ = plt.hist((FL_t0[s1]+FL_t1[s1])/2.,bins=NBINS, alpha=0.5, label='sector001')
_  = plt.hist((FL_t0[s2]+FL_t1[s2])/2.,bins=NBINS, alpha=0.5, label='sector002')
plt.plot([0, max(FL_t1)], [FLCUT, FLCUT], c='k')

# plt.title(str(np.size(FL_t0))+' flare candidates',fontsize=12)
plt.xlabel('epoch number')
plt.ylabel('# stars')
plt.legend(fontsize=12)
plt.yscale('log')


In [ ]:
flare_out2 = pd.DataFrame(data={'TIC':ALL_TIC[FL_id], 'i0':FL_t0, 'i1':FL_t1, 
                                'med':FL_f0, 'peak':FL_f1, 'FLOK':OK})
fdata = pd.merge(flare_out2, df, left_on='TIC', right_on='ticid')

In [ ]:
Fok = np.where(np.isfinite(fdata[u'parallax']) & # this is basically the same as the TGAS file...
               (fdata[u'parallax_error'] < 0.1) &
               (fdata[u'modality_flag'] == 1) & 
               (fdata[u'result_flag'] == 1) &
               np.isfinite(fdata[u'bp_rp']) & 
               (fdata[u'phot_bp_mean_flux_error']/fdata[u'phot_bp_mean_flux'] < 0.01) & 
               (fdata[u'phot_rp_mean_flux_error']/fdata[u'phot_rp_mean_flux'] < 0.01) & 
               (fdata[u'phot_g_mean_flux_error']/fdata[u'phot_g_mean_flux'] < 0.01) & 
               (fdata['FLOK'] == True) & 
               (fdata['peak']/fdata['med'] > 0.02))[0]
print(Fok.shape)


plt.figure(figsize=(9,9))

plt.scatter(fdata['bp_rp'][Fok], fdata['phot_g_mean_mag'][Fok] - 5. * np.log10(fdata[u'r_est'].values[Fok]) + 5, 
            alpha=0.6, s=9)

# cb = plt.colorbar()
# cb.set_label('LS Period (days)')
plt.xlabel('$G_{BP} - G_{RP}$ (mag)')
plt.ylabel('$M_G$ (mag)')
# plt.title('TESS Sectors 1,2', fontsize=12)
plt.ylim(15,-5)
# plt.savefig('cmd_per2.jpeg', dpi=150, bbox_inches='tight', pad_inches=0.25)